In [41]:
import yfinance as yf
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from bokeh.io import export_png, export_svgs
from bokeh.models import ColumnDataSource, DataTable, TableColumn

Нужно добавить максимальное количество русских акций и сделать словарь

In [50]:
class Prediction:
    tickers_list = {"лента":'LNTA.ME',"магнит":'MGNT.ME',"мтс": 'MTSS.ME',"яндекс": 'YNDX.ME',"сбербанк": 'SBER.ME' } 
    def __init__(self, name):
        print("New pred")
        self.columns=[]
        self.stock = tickers_list.get(name)
        self.columns.append(self.stock)
        self.now = datetime.datetime.now()
        print(self.stock)
    def prepare_data(self):
        datas = pd.DataFrame(columns=self.columns)
        delta_ = timedelta(days=1400)
        start_time=self.now - delta_
        delta_= timedelta(days=30)
        end_time =self.now + delta_
        post_start=start_time.strftime("%Y-%m-%d")
        post_end = end_time.strftime("%Y-%m-%d")
        for ticker in self.columns:
            datas[ticker] = yf.download(ticker,post_start,post_end)['Adj Close']
        datas= datas['LNTA.ME']
        past_days= 60
        fut_days= 7
        start= past_days
        end= len(datas)-fut_days
        cnt=end-start
        print("Рассматриваем дни с {} по {} всего {} примеров".format(start,end,cnt))
        past=[]
        fut=[]
        for i in range(start, end):
            past_data= datas[(i-past_days):i]
            future_data = datas[i:(i+fut_days)]
            past.append(list(past_data))
            fut.append(list(future_data))
        past_col=[]
        for i in range(past_days):
            past_col.append(f"past_{i}")
        fut_col=[]
        for i in range(fut_days):
            fut_col.append(f"fut_{i}")
        X= pd.DataFrame(past, columns=past_col)
        y= pd.DataFrame(fut, columns=fut_col)
        self.X_train = X[:-10]
        self.y_train = y[:-10]
        self.X_test = X[-10:]
    def predict(self):
        model = RandomForestRegressor()
        model.fit(self.X_train, self.y_train)
        self.prediction = model.predict(self.X_test)
        print(self.prediction)
    def make_table(self):
        day_ = timedelta(days=1)
        dates=[]
        for i in range(7):
            self.now=self.now+day_
            dates.append(self.now.strftime("%Y-%m-%d"))
        list_of_tuples = list(zip(dates, list(self.prediction[6])))
        self.made_df = pd.DataFrame(list_of_tuples, columns = ['Date', 'Cost, rubles'])
        self.made_df.set_index('Date', inplace=True)
    def make_png(self):
        source = ColumnDataSource(self.made_df)
        df_columns = [self.made_df.index.name]
        df_columns.extend(self.made_df.columns.values)
        columns_for_table=[]
        for column in df_columns:
            columns_for_table.append(TableColumn(field=column, title=column))
        data_table = DataTable(source=source, columns=columns_for_table,height_policy="auto",width_policy="auto",index_position=None)
        export_png(data_table, filename = 'new.png')

In [51]:
P=Prediction('лента')
P.prepare_data()
P.predict()
P.make_table()
P.make_png()

New pred
LNTA.ME
[*********************100%***********************]  1 of 1 completed
               LNTA.ME
Date                  
2017-06-26  340.500000
2017-06-27  350.000000
2017-06-28  350.000000
2017-06-29  347.500000
2017-06-30  346.500000
...                ...
2021-04-20  237.300003
2021-04-21  238.300003
2021-04-22  238.199997
2021-04-23  241.500000
2021-04-26  236.300003

[964 rows x 1 columns]
Рассматриваем дни с 60 по 957 всего 897 примеров
[[245.62299667 245.68199875 245.34700089 244.76100174 243.25099991
  241.74999954 239.28699875]
 [245.87399719 245.40999924 245.03200027 244.57100143 243.19700027
  241.97299973 239.68499863]
 [246.09599792 245.24699844 245.54599945 245.23900131 243.8980011
  242.47999985 240.15699844]
 [245.98799789 245.46199829 245.46799988 244.86700104 244.09500107
  242.67499969 240.10199921]
 [244.10499832 243.20199844 242.33099899 241.73700134 240.39500092
  239.3409996  237.55199921]
 [243.09899887 241.7019986  239.90599991 237.8830011  235.93100